In [77]:
import tensorflow_hub as hub
import pandas as pd
from numpy import linalg as LA
import numpy as np
import tensorflow as tf
import math

In [2]:
sess = tf.Session()

In [3]:
with open("/Users/zetong/Polisis/Facebook_privacy_entities.csv", mode = 'r', encoding = 'ascii', errors = 'ignore') as csvfile:
    entities = pd.read_csv(csvfile, header=None)
    texts = list(entities[0])
    labels = list(entities[1])

In [4]:
entities

,0,1
0,Who can send you friend requests?,1
1,Who can see your friends list?,1
2,Who can look you up using the email address yo...,1
3,Who can look you up using the phone number you...,1
4,Do you want search engines outside of Facebook...,1
5,Who can see your future posts?,2
6,Limit The Audience for Old Posts on Your Timel...,2
7,Review all your posts and things you're tagged...,2
8,Who can post on your timeline?,3
9,Who can see what others post on your timeline?,3


In [7]:
embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder/2")

In [14]:
sess.run([tf.global_variables_initializer(), tf.tables_initializer()])

[None, None]

In [19]:
X = sess.run(embed(texts))

In [67]:
def get_top_n_matches(texts, sim, n):
    return list(np.array(texts)[sorted(range(len(sim)), key=lambda i: sim[i], reverse=True)[:n]])

In [74]:
def sentence_embedding(sentence, sess):
    return sess.run(embed([sentence]))

## Cosine similarity

In [75]:
query = sentence_embedding("timeline setting", sess)
sim = []
for i in range(X.shape[0]):
    sim.append(np.dot(np.reshape(X[i], (1, X.shape[1])), np.transpose(query)) / (LA.norm(X[i], axis=0) * LA.norm(query, axis=1)))
get_top_n_matches(texts, sim, 3)

['Review what other people see on your timeline',
 'Who can see what others post on your timeline?',
 'Who can post on your timeline?']

## Angular distance similarity
from the paper: Universal Sentence Encoder (Cer et al., 2018): We find that using a similarity based on angular distance performs better on average than raw cosine similarity.

In [ ]:
query = sentence_embedding("audience setting", sess)
sim = []
for i in range(X.shape[0]):
    cos_sim = np.dot(np.reshape(X[i], (1, X.shape[1])), np.transpose(query)) / (LA.norm(X[i], axis=0) * LA.norm(query, axis=1))
    sim.append(1 - np.arccos(cos_sim)/math.pi)
get_top_n_matches(texts, sim, 3)